In [4]:
from dotenv import load_dotenv
import os

load_dotenv()  # Load variables from .env
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
# Import Langchain modules
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

# Other modules and packages
import os
# import tempfile
import streamlit as st  
import pandas as pd

/Users/szymonkrawczyk/Documents/code/repos/ocr-circus/venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY)
llm.invoke("Tell me a joke about cats")

AIMessage(content='Why was the cat sitting on the computer? \n\nBecause it wanted to keep an eye on the mouse!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 13, 'total_tokens': 35, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None}, id='run-fe143937-fd48-491e-a05e-fbfad8b0c525-0', usage_metadata={'input_tokens': 13, 'output_tokens': 22, 'total_tokens': 35, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
loader = PyPDFLoader("data/Oppenheimer-2006-Applied_Cognitive_Psychology.pdf")
pages = loader.load()
pages

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500,
                                            chunk_overlap=200,
                                            length_function=len,
                                            separators=["\n\n", "\n", " "])
chunks = text_splitter.split_documents(pages)

In [45]:
# Prompt template
PROMPT_TEMPLATE_BASE = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer
the question. If you don't know the answer, say that you
don't know. DON'T MAKE UP ANYTHING.

{context}

---

Answer the question based on the above context: {question}
"""

PROMPT_TEMPLATE = """
You are an expert PDF OCR and markdown conversion assistant.
Given an image of full page scanned spanish textbook:
1. Perform full OCR processing
2. Preserve original document structure
3. Convert to clean markdown
4. Maintain:
   - Original formatting
   - Section hierarchies
   - Typography distinctions
   - Tables and lists
5. Clean up OCR artifacts
6. Ensure maximum text accuracy
7. Handle multi-column layouts intelligently
8. Detect and properly format headers, paragraphs, captions

Output requirements:
- Fully searchable markdown file
- Professional formatting
- No OCR noise/errors
- Semantic markdown structure
- Readable and well-organized
"""


# 9. If you encounter photos or ilustrations, describe them.

In [ ]:
# Concatenate context text
context_text = "\n\n---\n\n".join([doc.page_content for doc in relevant_chunks])

# Create prompt
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, 
                                question="What is the title of the paper?")
print(prompt)

In [ ]:
llm.invoke(prompt)


In [ ]:
# Using Langchain Expression Language
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
            {"context": retriever | format_docs, "question": RunnablePassthrough()}
            | prompt_template
            | llm
        )
rag_chain.invoke("What's the title of this paper?")

In [ ]:
# Generate structured responses


## vision test

In [6]:
import base64
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [26]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.messages import HumanMessage, SystemMessage
# gpt-4-turbo
chat = ChatOpenAI(model="gpt-4o-mini", max_tokens=1000)
def send_chat(chat,prompt,encoded_image):
    res=chat.invoke(
    [
        HumanMessage(
            content=[
                {"type": "text", "text": prompt},
                {
                    "type": "image_url",
                    # "image_url": {
                    #     "url": "https://raw.githubusercontent.com/langchain-ai/langchain/master/docs/static/img/langchain_stack.png",
                    #     "detail": "auto",
                    # },
                    # "image_url": f"data:image/png;base64,{base64_image}"
                    "image_url": {"url": f"data:image/png;base64,{encoded_image}"}
                },
            ]
        )
    ]
    )
    return res


# bicycle example cost in 591 out 60 = 651
# big page in 1000 out 584 = 1584 11sec

In [11]:
# encoded_image = encode_image("data/boybicycle.jpg")
encoded_image = encode_image("data/ai1p103.jpeg")

In [46]:
# prompt="What is this image showing"
prompt=PROMPT_TEMPLATE

In [47]:
response=send_chat(chat,prompt,encoded_image)

In [48]:
response.dict()

/var/folders/wj/hsg00rb90b76klzxn2nt3sk40000gn/T/ipykernel_21493/3202056457.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  response.dict()


{'content': '```markdown\n# PRACTICAR Y COMUNICAR\n\n## 7. MADRID\n**P:** 165, **Ej. 1:** P 167, **Ej. 14 y 16**\n\nA. Estos son tres barrios de Madrid. ¿En qué barrio crees que vive cada persona?\n\n### BARRIOS EMBLEMÁTICOS DE MADRID\n\n**Lavapiés** está en el centro de Madrid. Es un barrio bohemio, antiguo y con pocos comodidades, pero con mucho encanto. Las calles son estrechas y hay muchos bares. En general, los alquileres no son muy caros y por eso muchos artistas y jóvenes viven aquí. En este barrio viven también muchos inmigrantes y gente mayor. En Lavapiés hay bastantes corrales, bloques de pisos pequeños con un patio interior comunitario.\n\n**Chamberí** es un barrio céntrico y bastante elegante. En la actualidad es uno de los barrios más caros de Madrid, con pisos grandes en edificios de principios del siglo xx. Tiene zonas peatonales, tiendas de todo tipo, gimnasios, cines... También hay muchos bares y restaurantes y es uno de los mejores barrios de Madrid para ir de tapas y

In [49]:
response.content

'```markdown\n# PRACTICAR Y COMUNICAR\n\n## 7. MADRID\n**P:** 165, **Ej. 1:** P 167, **Ej. 14 y 16**\n\nA. Estos son tres barrios de Madrid. ¿En qué barrio crees que vive cada persona?\n\n### BARRIOS EMBLEMÁTICOS DE MADRID\n\n**Lavapiés** está en el centro de Madrid. Es un barrio bohemio, antiguo y con pocos comodidades, pero con mucho encanto. Las calles son estrechas y hay muchos bares. En general, los alquileres no son muy caros y por eso muchos artistas y jóvenes viven aquí. En este barrio viven también muchos inmigrantes y gente mayor. En Lavapiés hay bastantes corrales, bloques de pisos pequeños con un patio interior comunitario.\n\n**Chamberí** es un barrio céntrico y bastante elegante. En la actualidad es uno de los barrios más caros de Madrid, con pisos grandes en edificios de principios del siglo xx. Tiene zonas peatonales, tiendas de todo tipo, gimnasios, cines... También hay muchos bares y restaurantes y es uno de los mejores barrios de Madrid para ir de tapas y salir de no

In [50]:
def save_as_markdown(content, file_path):
    """
    Saves the given content as a markdown file after trimming the first and last lines.

    Args:
        content (str): The content to be saved.
        file_path (str): The path where the markdown file will be saved.

    Returns:
        None
    """
    lines = content.split('\n')
    trimmed_content = '\n'.join(lines[1:-2])
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(trimmed_content)

# Example usage
markdown_content = response.content
file_path = 'output.md'
save_as_markdown(markdown_content, file_path)